In [36]:
!python3 main.py play --agents q_agent rule_based_agent rule_based_agent rule_based_agent --train 1 --scenario clean --n-rounds 10 --turn-based

  0%|                                                    | 0/10 [00:00<?, ?it/s]coord [(11, 15), (14, 1), (5, 4), (2, 13), (11, 8), (13, 13), (14, 5), (12, 1), (3, 15), (5, 11), (13, 2), (13, 10), (1, 5), (6, 13), (6, 15), (14, 15), (3, 9), (8, 13), (7, 5), (13, 5), (1, 11), (15, 10), (9, 4), (13, 14), (11, 10), (3, 3), (5, 8), (7, 12), (15, 12), (4, 11), (1, 4), (3, 11), (4, 5)]
coins [(13, 13), (14, 15), (13, 14), (15, 12)]
17
17
events ['BOMB_DROPPED']
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (15, 15)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, 1, -1, -1, 1]
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (15, 15)
d [1. 1.]
max d i j [0 1] 0
features [2, 0, 1, -1, -1, 2]
[0. 0. 0. 0. 0. 0.] 
old q 0.0
new q -0.7000000000000001
coord [(11, 15), (14, 1), (5, 4), (2, 13), (11, 8), (13, 13), (14, 5), (1

coord [(15, 3), (12, 15), (3, 1), (5, 13), (11, 15), (5, 4), (2, 13), (11, 8), (13, 13), (14, 5), (5, 11), (13, 10), (6, 13), (6, 15), (3, 9), (8, 13), (7, 5), (13, 5), (1, 11), (15, 10), (9, 4), (11, 10), (3, 3), (5, 8), (7, 12), (15, 12), (4, 11), (3, 11)]
coins [(12, 15), (11, 15), (13, 13), (15, 12)]
13
13
events ['INVALID_ACTION']
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (13, 14)
d [2.]
max d i j [0] 3
features [2, 0, -1, 2, -1, 2]
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (13, 14)
d [2.]
max d i j [0] 3
features [2, 0, -1, 2, -1, 2]
[0. 0. 0. 0. 0. 0.] 
old q 0.0
new q -10.0
coord [(15, 3), (12, 15), (3, 1), (5, 13), (11, 15), (5, 4), (2, 13), (11, 8), (13, 13), (14, 5), (5, 11), (13, 10), (6, 13), (6, 15), (3, 9), (8, 13), (7, 5), (13, 5), (1, 11), (15, 10), (9, 4), (11, 10), (3, 3), (5, 8), (7, 12), (15, 12), (4, 11), (3, 11)]
coins [(12, 15), (11, 15), (13, 13), (15, 12)]
13
13
events ['MOVED_UP', 'CO

coord [(11, 3), (3, 14), (3, 11), (11, 14), (7, 4), (4, 3), (7, 13), (8, 13), (9, 5), (7, 11), (5, 5), (9, 7), (7, 8), (1, 7), (11, 9), (2, 9), (6, 7), (7, 12), (9, 9), (5, 7), (6, 1), (10, 13), (10, 5), (13, 6), (9, 6)]
coins [(4, 3)]
15
15
events ['INVALID_ACTION']
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (3, 2)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, -1, 1, -1, 0]
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (3, 2)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, -1, 1, -1, 0]
[  1.9   -18.829  -0.6     0.      0.      0.   ] 
old q -18.829
new q -26.775100000000002
coord [(11, 3), (3, 14), (3, 11), (11, 14), (7, 4), (4, 3), (7, 13), (8, 13), (9, 5), (7, 11), (5, 5), (9, 7), (7, 8), 

features [2, 0, 2, 1, 2, 0]
 20%|████████▊                                   | 2/10 [01:32<05:15, 39.42s/it]coord [(14, 1), (5, 5), (9, 9), (13, 2), (11, 7), (4, 9), (3, 14), (13, 13), (6, 5), (3, 6), (11, 9), (11, 3), (10, 1), (6, 9), (12, 5), (13, 3), (10, 9), (6, 15), (1, 5), (11, 4), (5, 6), (13, 4), (15, 9), (4, 15), (13, 10), (15, 6), (5, 10), (5, 8), (12, 7), (11, 8), (13, 5)]
coins [(3, 14), (4, 15)]
19
19
events ['INVALID_ACTION']
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (1, 15)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, -1, -1, 1, 0]
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (1, 15)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, -1, -1, 1, 0]
[0. 0. 0. 0. 0. 0.] 
old q 0.0
n

coord [(11, 13), (3, 9), (2, 5), (8, 3), (15, 9), (6, 13), (15, 12), (8, 11), (11, 7), (11, 12), (15, 8), (3, 5), (11, 11), (13, 10), (9, 6), (3, 2), (11, 5), (12, 3), (11, 6), (6, 1), (15, 11), (9, 3), (15, 5), (6, 3), (11, 3), (10, 13), (5, 3), (10, 3), (5, 5)]
coins [(12, 3)]
18
18
events ['MOVED_RIGHT', 'COIN_DISTANCE_INCREASED', 'CRATE_DISTANCE_INCREASED']
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (14, 1)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, 1, -1, 0, -1]
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (15, 1)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [0, 0, 0, 1, -1, -1]
[  3.61     -26.7751    -1.529759 -18.5212    -3.3212    -0.466   ] 
old q -1.529759
new q -2.3665421
coord [(11

...

j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, -1, 1, 1, -1]
[ 3.235     1.953091 -9.70885  -9.8101    0.       -0.7     ] 
old q -9.8101
new q -18.53794
coord [(10, 1), (7, 4), (7, 13), (11, 9), (2, 11), (5, 12), (15, 3), (6, 7), (4, 9), (3, 6), (3, 11), (10, 9), (7, 15), (9, 1), (3, 12), (11, 5), (11, 15), (7, 1), (6, 5), (9, 14), (14, 3), (8, 3), (14, 9), (5, 8), (3, 10), (14, 7), (9, 13), (7, 7), (9, 4), (11, 3), (13, 7), (1, 8), (15, 15), (1, 3), (6, 3), (10, 11), (15, 11), (10, 5)]
coins [(1, 3)]
8
8
events ['WAITED']
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (1, 1)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, -1, 1, 1, -1]
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (1, 1)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosi

coord [(3, 13), (7, 4), (11, 9), (2, 11), (5, 12), (15, 3), (6, 7), (4, 9), (3, 6), (3, 11), (10, 9), (3, 12), (11, 5), (11, 15), (6, 5), (9, 14), (14, 3), (8, 3), (14, 9), (5, 8), (3, 10), (14, 7), (7, 7), (9, 4), (11, 3), (13, 7), (1, 8), (1, 3), (6, 3), (10, 11), (15, 11), (10, 5)]
coins [(1, 3)]
6
5
events ['MOVED_UP', 'COIN_DISTANCE_INCREASED', 'CRATE_DISTANCE_INCREASED']
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (1, 2)
d [1. 1.]
max d i j [0 1] 1
features [2, 0, -1, 1, -1, 2]
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (1, 1)
features [2, 0, -1, 2, 1, -1]
[  1.9    -18.5554   2.6     -9.766   -3.3554  -9.766 ] 
old q 1.9000000000000001
new q 1.6810000000000003
coord [(5, 1), (3, 13), (11, 9), (2, 11), (5, 12), (15, 3), (6, 7), (4, 9), (3, 6), (3, 11), (10, 9), (3, 12), (11, 5), (11, 15), (6, 5), (14, 3), (8, 3), (14, 9), (5, 8), (3, 10), (14, 7), (7, 7), (9, 4), (11, 3), (13, 7), (1, 8), (1, 3), (6, 3), (10

j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, 1, -1, 1, -1]
[ 10.05115   -18.0953965   2.43091     0.         -1.0953965   0.       ] 
old q 0.0
new q -9.0953965
coord [(5, 1), (3, 13), (11, 9), (2, 11), (5, 12), (15, 3), (6, 7), (4, 9), (3, 6), (3, 11), (10, 9), (3, 12), (6, 5), (14, 3), (5, 8), (3, 10), (14, 7), (7, 7), (11, 3), (1, 8), (1, 3)]
coins [(5, 1), (1, 3)]
5
5
events ['WAITED']
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (2, 1)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, 1, -1, 1, -1]
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (2, 1)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, 1, -1, 1, -1]
[ 10.05115   -18.0953965   2.43091    -9.0

coord [(6, 9), (7, 12), (5, 2), (11, 5), (2, 1), (7, 13), (13, 14), (5, 6), (3, 7), (5, 14), (8, 9), (6, 15), (5, 5), (3, 6), (12, 3), (8, 1), (7, 3), (9, 4), (12, 1), (9, 3), (2, 7), (7, 1), (9, 7), (11, 7)]
coins [(2, 1)]
22
22
events ['MOVED_DOWN', 'COIN_DISTANCE_INCREASED', 'CRATE_DISTANCE_REDUCED']
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (1, 1)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, -1, 0, 1, -1]
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (1, 2)
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
features [3, 0, -1, 1, -1, 1]
[ 0.          1.28240287 -9.88458374  0.          0.          0.        ] 
old q 1.2824028733900004
new q 2.3947126353764596
coord [(6, 9), (7, 12), (5, 2), (11, 5), (2, 1),

coord [(11, 15), (5, 13), (6, 9), (7, 12), (5, 2), (2, 1), (7, 13), (5, 6), (3, 7), (8, 9), (5, 5), (3, 6), (12, 3), (8, 1), (7, 3), (9, 4), (12, 1), (9, 3), (2, 7), (7, 1), (9, 7)]
coins [(2, 1)]
19
19
events ['INVALID_ACTION']
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (1, 3)
d [1. 1. 1.]
max d i j [0 1 2] 1
features [2, 0, -1, 1, 2, 2]
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (1, 3)
d [1. 1. 1.]
max d i j [0 1 2] 1
features [2, 0, -1, 1, 2, 2]
[0. 0. 0. 0. 0. 0.] 
old q 0.0
new q -10.0
coord [(5, 13), (6, 9), (7, 12), (5, 2), (2, 1), (7, 13), (5, 6), (3, 7), (8, 9), (5, 5), (3, 6), (12, 3), (8, 1), (7, 3), (9, 4), (12, 1), (9, 3), (2, 7), (7, 1), (9, 7)]
coins [(2, 1)]
19
19
events ['MOVED_RIGHT', 'COIN_DISTANCE_REDUCED', 'CRATE_DISTANCE_REDUCED']
j explosion 0 0.0
j explosion 1 0.0
j explosion 2 0.0
j explosion 3 0.0
my_pos  (1, 3)
d [1. 1. 1.]
max d i j [0 1 2] 1
features [2, 0, -1, 1, 2, 2]
j explosion 0 

In [3]:
!python3 main.py play --agents q_agent --scenario classic --n-rounds 5 --turn-based

nonzero rows: 114
  0%|                                                     | 0/5 [00:00<?, ?it/s]my_pos  (15, 1)
n_values 1  [0, 0, -1, -1]
features [0, 0, 0, 1, -1, -1]
\state, i, r, a [ 0  0  0  1 -1 -1] 80 2 5
model [  -9.51394399  -12.1315094  -106.50836292 -104.90637425  -24.90639154
   -3.00187849] 

BOMB
my_pos  (15, 1)
n_values 1  [0, 0, -1, -1]
features [2, 0, 1, 2, -1, -1]
\state, i, r, a [ 2  0  1  2 -1 -1] 152 2 1
model [ -5.69674016   7.06462261 -78.52104725 -78.8288797  -15.26028148
 -77.75853009] 

LEFT
my_pos  (14, 1)
n_values 1  [0, -1, 2, -1]
features [2, 0, 1, -1, 2, -1]
\state, i, r, a [ 2  0  1 -1  2 -1] 424 3 2
model [-19.         -10.           0.5322451  -45.02016947  -5.23923351
 -39.38168074] 

LEFT
my_pos  (13, 1)
n_values 1  [0, 0, 0, -1]
features [2, 0, 1, 2, 2, -1]
\state, i, r, a [ 2  0  1  2  2 -1] 568 3 2
model [ -4.47053748  -5.99947353   2.31359816  -9.70477118  -3.8
 -34.76096453] 

LEFT
my_pos  (12, 1)
n_values 1  [0, -1, 0, -1]
features [2, 0, 1, 

my_pos  (3, 15)
n_values 1  [0, -1, 0, 0]
features [0, 0, 0, -1, 0, 1]
\state, i, r, a [ 0  0  0 -1  0  1] 304 1 2
model [ -52.29751025  -50.86478705  -48.99095845 -122.64065354  -56.15230151
  -50.56878926] 

RIGHT
my_pos  (4, 15)
n_values 1  [0, -1, 0, -1]
features [0, 0, 0, -1, 1, -1]
\state, i, r, a [ 0  0  0 -1  1 -1] 224 3 2
model [ -61.8281586  -105.3233591     0.         -111.05003364  -54.12088091
  -49.11283186] 

LEFT
my_pos  (3, 15)
n_values 1  [0, -1, 0, 0]
features [0, 0, 0, -1, 0, 1]
\state, i, r, a [ 0  0  0 -1  0  1] 304 1 2
model [ -52.29751025  -50.86478705  -48.99095845 -122.64065354  -56.15230151
  -50.56878926] 

RIGHT
my_pos  (4, 15)
n_values 1  [0, -1, 0, -1]
features [0, 0, 0, -1, 1, -1]
\state, i, r, a [ 0  0  0 -1  1 -1] 224 3 2
model [ -61.8281586  -105.3233591     0.         -111.05003364  -54.12088091
  -49.11283186] 

LEFT
my_pos  (3, 15)
n_values 1  [0, -1, 0, 0]
features [0, 0, 0, -1, 0, 1]
\state, i, r, a [ 0  0  0 -1  0  1] 304 1 2
model [ -52.2975102

In [38]:
#--my-agents --save-replay --turn-based --save-stats

In [ ]:
!python main.py replay "replays/Round 07 (2022-03-23 12-52-13).pt" --turn-based

In [ ]:
reward = np.load("agent_code/q_agent/rewards.npy")
plt.plot(reward,'.')
plt.xlabel("rounds")
plt.ylabel("reward")
#plt.ylim(-100,100)

In [ ]:
model_trained = np.load("agent_code/q_agent/my-saved-model.npy")

In [17]:
import itertools
import numpy as np

v = [0,0, 1, -1,  1, -1]

NEIGHBOURING_FIELDS = [-1,0,1,2] # the possible values for neighbouring fields
GAME_MODE = [0,1,2,3] # the possible values for game mode
CURRENT_FIELD = [0,1,2,3] # the possible values for current filed
dic = {}

def get_arrangements(array):
    return [ np.roll(array, -i) for i in range(0,4) ]

# get all possible combinations
def get_variations(array, r=4):
    return [ item for item in list(itertools.product(array, repeat=r)) ]

# compute state to index for the Q-table
def build_state_to_index(arr1 = NEIGHBOURING_FIELDS, arr2 = GAME_MODE, arr3 = CURRENT_FIELD):
    i = 0
    perm = get_variations(arr1)
    comb = np.array(np.meshgrid(arr2, arr3)).T.reshape(-1,2) # combinations of arr2 and arr3
    step = len(comb)
    for p in perm:
        already_there = []
        arrangements = get_arrangements(p)
        for a in arrangements:
            if tuple(a) in dic.keys():
                already_there.append(True)
            else:
                already_there.append(False)
        if sum(already_there) == 0: # p not in dictionary
            value = [v for v in range(i, i+step)] # value indices for p
            i += step # update i
            dic.update({tuple(p) : value}) # add in dictionary

# give corresponding index and rotation from keys
def get_state_index(state):
    arrangements = get_arrangements(state[-4:]) # rotations of neighbouring fields
    mode_and_field = state[:2]
    value_index = 0
    comb = np.array(np.meshgrid(GAME_MODE, CURRENT_FIELD)).T.reshape(-1,2)
    for i, c in enumerate(comb):
        if (c == mode_and_field).all():
            value_index = i
            break
    for j, a in enumerate(arrangements):
        if tuple(a) in dic.keys():
            print(tuple(a), value_index)
            return dic[tuple(a)][value_index], j
    assert True == 0, state
            
build_state_to_index()
print(get_arrangements(v))
print(get_state_index(v))

[array([ 0,  0,  1, -1,  1, -1]), array([ 0,  1, -1,  1, -1,  0]), array([ 1, -1,  1, -1,  0,  0]), array([-1,  1, -1,  0,  0,  1])]
(-1, 1, -1, 1) 0
(400, 1)


In [5]:
import numpy as np
import random


In [51]:
def dist(pos, objects):
    return np.sqrt( np.power(np.subtract(objects, pos).transpose()[0], 2) + np.power(np.subtract(objects, pos).transpose()[1], 2) )

p = (1,1)
k = [[0,2], [1,2], [1,3], [2,1], [2,1], [1,1]]

distances = np.array(dist(p,k))
print(distances)
print(np.where(distances == min(distances)))

print(dist((0,0), (0,0)))

[1.41421356 1.         2.         1.         1.         0.        ]
(array([5]),)
0.0


In [36]:
pos = (0,0)
def get_neighbours(pos):
    sub = [(0,-1), (-1,0), (-2,0), (0,1)] # left, down, right, up

    neighbours = []
    for i in sub:
        neighbour = np.subtract(pos, i)
        if (0 <= neighbour[0] < 17) and (0 <= neighbour[1] < 17): # check if in game borders
            neighbours.append(neighbour)
    return neighbours

neigh = get_neighbours((1,1))
print(neigh)
neigh.append(np.array(pos))
print(neigh)

a = neigh[0]
print(a)
b = (a[0], a[1])
b = np.array(b)
neigh.append(b)
print(neigh)

[array([1, 2]), array([2, 1]), array([3, 1]), array([1, 0])]
[array([1, 2]), array([2, 1]), array([3, 1]), array([1, 0]), array([0, 0])]
[1 2]
[array([1, 2]), array([2, 1]), array([3, 1]), array([1, 0]), array([0, 0]), array([1, 2])]


In [27]:
pos = np.array((1,5))
area = [[1, 3], [1, 4], [1, 5], [1, 2]]
i = -1
for a in area:
    print(a, pos)
    i += 1
    if ((a == pos).all()):
        print(a, pos)
        break
print(i)

[1, 3] [1 5]
[1, 4] [1 5]
[1, 5] [1 5]
[1, 5] [1 5]
2


In [22]:
pos = (1,2)
pos1 = (1,2)
a = [1,2]
b = ((1,3), (1,2), (2,2))
c = any([(pos==b_) for b_ in b])

print(c)
print(pos!=b[2])
print(pos!=pos1)

True
True
False


In [23]:
mask = [[0]]
n = [[0,1],[1,2],[2,1],[1,0]]


In [24]:
n.append([pos[0], pos[1]])
print(n)

[[0, 1], [1, 2], [2, 1], [1, 0], [1, 2]]


In [49]:
a = np.array([5,6,3,2,1,1])
print(min(a))
i = np.where(a==min(a))
print(a[i])
print(1 in a[i])

1
[1 1]
True


In [62]:
A = np.array([[1, 4], [2, 5], [3, 6]])
B = np.array([[1, 4], [3, 6], [7, 8]])
aset = set([tuple(x) for x in A])
bset = set([tuple(x) for x in B])
intersection = np.array([x for x in aset & bset])
print(intersection)

[[1 4]
 [3 6]]


In [68]:
list1 = [(14, 3),(23, 41),(33, 62),(1, 3),(3, 3)]
list2 = [(14, 3),(23, 41),(34, 62),(2, 3),(5, 5)]
list3 = [item for item in list1 if item in list2]
print(list3)

[(14, 3), (23, 41)]


In [150]:
a = np.array([1, 3, 1, 7])
c = np.array([[0,1],[1,0],[0,0],[1,1]])
b = np.where(a!=1)
print(c)
d = [[0,1], [1,0], [0,0]]

k = np.where([d_ not in c for d_ in d])[0]
print(np.empty(k))

[[0 1]
 [1 0]
 [0 0]
 [1 1]]
1.8483715e-316


In [19]:
a = (1,1)
b = (1,1)
print(a!=b)

False


In [68]:
from datetime import datetime

now = datetime.now() # current date and time

year = now.strftime("%Y")
print("year:", year)

month = now.strftime("%m")
print("month:", month)

day = now.strftime("%d")
print("day:", day)

time = now.strftime("%H:%M:%S")
print("time:", time)

date_time = now.strftime("%H_%M_%S")
print("date and time:",date_time)

year: 2022
month: 03
day: 23
time: 18:59:01
date and time: 18_59_01


In [ ]:
import time
start = time.time()

time.sleep(5)
end  = time.time()
print(end-start)